In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as plt

DIR = "Master_Loan_Summary.csv"

In [ ]:
ds = pd.read_csv(DIR)
#Taking out useless variables
ds = ds.drop(["loan_number", "term", "grade", "origination_date", "listing_title", "last_payment_date", "next_payment_due_date", "data_source"], axis = 1)
ds.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,amount_borrowed,borrower_rate,installment,principal_balance,principal_paid,interest_paid,late_fees_paid,debt_sale_proceeds_received,days_past_due,loan_status_description
0,27050.0,0.1099,885.46,0.0,27050.0,4702.53,0.0,0.0,0,COMPLETED
1,4800.0,0.1099,157.13,0.0,4800.0,357.52,0.0,0.0,0,COMPLETED
2,12000.0,0.0762,373.94,0.0,12000.0,1397.54,0.0,0.0,0,COMPLETED
3,12000.0,0.1199,398.52,0.0,12000.0,2346.48,0.0,0.0,0,COMPLETED
4,12000.0,0.0662,368.45,0.0,12000.0,1263.95,0.0,0.0,0,COMPLETED


In [ ]:

#Creating a dataset for every Grade
status = {"CHARGEOFF": 1,"COMPLETED": 0,"CURRENT": None, "DEFAULTED": 1}
datasets = {}
for des, value in status.items():
    mask = ds["loan_status_description"].isin([des])
    datasets[des] = ds.loc[mask]
    if des != "CURRENT":
        datasets[des]["result"] = value
    datasets[des] = datasets[des].drop(["loan_status_description"], axis =1)
    


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
print(datasets["DEFAULTED"].sample())

         amount_borrowed  borrower_rate  ...  days_past_due  result
2171058          15000.0         0.1095  ...              0       1

[1 rows x 10 columns]


In [ ]:
#splitting inputs and outputs
all_dataset = pd.concat([datasets["CHARGEOFF"], datasets["DEFAULTED"], datasets["COMPLETED"]])
X = all_dataset.iloc[:, :-1].values
y = all_dataset.iloc[:, -1].values

In [ ]:
#splitting dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [ ]:
#scaling our values
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
X_train

array([[-0.96120624,  1.5637401 , -0.79738562, ..., -0.13931302,
        -0.20223961, -0.35802068],
       [ 1.07447544, -0.47022071,  1.30343548, ...,  3.28923691,
        -0.20223961,  0.53836444],
       [-0.16723069, -1.13831003, -0.14073774, ..., -0.13931302,
        -0.20223961, -0.35802068],
       ...,
       [ 0.11931688,  0.2943704 ,  0.37595059, ...,  1.04603217,
         2.25416437,  2.74307636],
       [-0.99105494,  0.03826949, -1.17542443, ..., -0.13931302,
        -0.20223961, -0.35802068],
       [-0.47765722, -0.80240957, -0.42999585, ..., -0.13931302,
        -0.20223961, -0.35802068]])

In [ ]:
#Training Model
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [ ]:
#Making a Confusion matrix with the validation set
y_pred = classifier.predict(X_test)
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[293370    312]
 [   298  78864]]


0.9983639270043235

In [ ]:
from termcolor import colored
#predicting one sample of the current loans
sample = datasets["CURRENT"].sample()

print("amount_borrowed: " , int(sample['amount_borrowed']))
print("borrower_rate: " , float(sample['borrower_rate']))
print("installment: " , int(sample['installment']))
print("principal_balance: " , int(sample['principal_balance']))
print("principal_paid: " , int(sample['principal_paid']))
print("interest_paid: " , int(sample['interest_paid']))
print("late_fees_paid: " , float(sample['late_fees_paid']))
print("debt_sale_proceeds_received: " ,float( sample['debt_sale_proceeds_received']))
print("days_past_due: " , int(sample['days_past_due']))
print("\n")
prediction = classifier.predict(sc.transform(sample))
if prediction[0] == 1:
    print(colored("Prediction: More likely to Default", 'red'))
else:
    print(colored("Prediction: More likely to Pay their Debts", 'green'))

amount_borrowed:  20000
borrower_rate:  0.0959
installment:  641
principal_balance:  16035
principal_paid:  3964
interest_paid:  1167
late_fees_paid:  0.0
debt_sale_proceeds_received:  0.0
days_past_due:  0


Prediction: More likely to Pay their Debts
